Imports

In [9]:
import feed_forward
import numpy as np
import torch
import torch.nn as nn
import geoopt
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

Training and Test data from MNIST data set

In [10]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

Initializing the model

In [11]:
model = feed_forward.HypFF()

In [12]:
print(model)

HypFF(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): MobLinear(
    in_features=784, out_features=512, bias=True
    (ball): PoincareBall manifold
  )
  (fc2): MobLinear(
    in_features=512, out_features=512, bias=True
    (ball): PoincareBall manifold
  )
  (fc3): MobLinear(
    in_features=512, out_features=10, bias=True
    (ball): PoincareBall manifold
  )
)


Hyperparameters

In [13]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Loss Function

In [22]:
loss_fn = nn.CrossEntropyLoss() #Initiliaze the loss function

Optimizer

In [23]:
optimizer = geoopt.optim.RiemannianSGD(model.parameters(), lr=learning_rate)

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = geoopt.PoincareBall.mobius_fn_apply(loss_fn, pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


AttributeError: 'CrossEntropyLoss' object has no attribute 'k'